In [77]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

#  options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#  Webscrape the Toronto Neighborhoods

In [51]:
#  list of postal codes of Toronto area contains all the information we need to assign:
#  
#      PostalCode, Borough, Neighborhood
#
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url)
soup = BeautifulSoup(html_data.text, 'html.parser')

In [52]:
#  how many tables does the webpage contain?
print('There are {} tables defined in the webpage'.format(len(soup.find_all('table'))))

There are 3 tables defined in the webpage


In [53]:
#  pick the table with first cell entry: M1A
table = [tbl for tbl in soup.find_all('table') if 'M1A' in tbl.find('tbody').find('tr').find('td').text]
table = table[0].find('tbody')

Scrape the table and make sure to identify multiple neighborhoods and comma-separate them. It seems by the code example given from the instructor, namely:

`(((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')`

that cases like **M3K** with an entry like `North York (Downsview) East (CFB Toronto)` should have the `Neighborhood` defined only as `Downsview East`, i.e. any extra parenthesized entry like `(CFB Toronto)` should be ignored.

In [54]:
toronto = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
for cell in table.find_all('td'):
    if cell.span.text == 'Not assigned':
        continue
    #  capture postal code, borough and all neighborhoods
    postalcode, borough, neighborhoods = re.search(r'(M\d[A-Z])([^(]+)\((.*)$', cell.text.strip()).groups()
    
    #  remove all extra parenthesized neighborhoods
    #  replace any leftover closed parenthesis with space
    #  replace any / with comma
    neighborhoods = neighborhoods.split('(')[0].strip(')').replace(')', ' ').replace(' /', ',').strip(' ')
    toronto.loc[len(toronto)] = [postalcode, borough, neighborhoods]

In [55]:
print(f'Total number of neighborhoods found = {len(toronto)}')

Total number of neighborhoods found = 103


Identify Boroughs with unusually long names. We might have to manually fix them.

In [56]:
toronto.loc[ toronto.Borough.str.len() > 20 ]

,PostalCode,Borough,Neighborhood
35,M4J,East YorkEast Toronto,The Danforth East
76,M7R,MississaugaCanada Post Gateway Processing Centre,Enclave of L4W
92,M5W,Downtown TorontoStn A PO Boxes25 The Esplanade,Enclave of M5E
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L


In [57]:
#  ok, let's manually fix them
tofix = toronto.loc[ toronto.Borough.str.len() > 20, 'PostalCode']
toronto.loc[ toronto.Borough.str.len() > 20, 'Borough' ] = ['East York',
                                                            'Mississauga',
                                                            'Downtown Toronto',
                                                            'East Toronto']
toronto[ toronto.PostalCode.isin(tofix) ]

,PostalCode,Borough,Neighborhood
35,M4J,East York,The Danforth East
76,M7R,Mississauga,Enclave of L4W
92,M5W,Downtown Toronto,Enclave of M5E
100,M7Y,East Toronto,Enclave of M4L


In [74]:
#  are there any Boroughs with 'Not assigned' neighborhoods?
toronto.query('Neighborhood.str.contains("[Nn]ot").values')

,PostalCode,Borough,Neighborhood


In [75]:
toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [76]:
toronto.shape

(103, 3)

# Geocode the Toronto Neighborhoods

In [85]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [86]:
#  get the Toronto latitude and longitude coordinates
gl = Nominatim(user_agent='toronto_neighborhoods')
toronto_gl = gl.geocode('Toronto, Ontario')
toronto_lat , toronto_lng = toronto_gl.latitude, toronto_gl.longitude
print(f'Toronto latitude = {toronto_lat} and longitude = {toronto_lng}')

Toronto latitude = 43.6534817 and longitude = -79.3839347


We will try to get the Toronto Neighborhood coordinates using the Google Maps API. We define the following function which tries 10 times before stopping. 

In [123]:
#  Now let's get the rest of the Toronto Neighborhood coordinates if we can from Google Maps
def assign_latlng(postalcode):
    coords = None
    STOP = 10
    while (coords is None):
        g = geocoder.google(f'{postalcode}, Toronto, Ontario')
        coords = g.latlng
        STOP -= 1
        if (STOP == 0):
            print('It does not work..')
            break
    return coords

Let's try for the first postalcode to see if it works.

In [124]:
assign_latlng(toronto.PostalCode[0])

It does not work..


Ok, then let's use the downloaded CSV file instead.

In [138]:
toronto_latlng = pd.read_csv('Geospatial_data', header=0)
toronto_latlng.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

#  there should not be any differences in Postal Codes between our webscrapped set and this one
assert( set(toronto_latlng.PostalCode) == set(toronto.PostalCode) )
toronto_latlng.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [141]:
#  inner join
toronto = toronto.merge(toronto_latlng, on='PostalCode')

In [142]:
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [143]:
toronto.shape

(103, 5)

#  Explore the Toronto Neighborhoods

Let's create first a map of all the neighborhoods.

In [144]:
import folium

In [158]:
#  shift location a bit so all neighborhoods fit at zoom level 11
toronto_map = folium.Map(location=[toronto_lat+0.06, toronto_lng+0.05], zoom_start=11)
for lat, lng, label in zip(toronto.Latitude, toronto.Longitude, toronto.Neighborhood):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
toronto_map

In [163]:
#  prepare to talk to the Foursquare API
CLIENT_ID = 'W3SH5A1VQH330RLC4U5J4XRRC3FYJYMBYDN4KIXV5CFBZNV5'
CLIENT_SECRET = 'XMHABSF1WWR0TZ2AJNM0POZ0FYSU0DDE4YWPS5DMVEHLRQUX'
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [179]:
#  borrow the wrapper function from the exercise lab
#  modify it when a given neighborhood request does not produce any results not to fail but keep going 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
            venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
            
        except:
            venues_list.append([(
            name, 
            lat, 
            lng, 
            None, 
            None, 
            None,  
            None)])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [180]:
#  query Foursquare for top 100 venues in each of the Toronto Neighborhoods
toronto_venues = getNearbyVenues(toronto.Neighborhood, toronto.Latitude, toronto.Longitude, radius=500)

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

Check if we have unfilled entries.

In [189]:
toronto_venues.applymap(lambda x: x is None).sum()

Neighborhood              0
Neighborhood Latitude     0
Neighborhood Longitude    0
Venue                     1
Venue Latitude            0
Venue Longitude           0
Venue Category            1
dtype: int64

Which neighborhood(s) did not get any values?

In [205]:
pick = toronto_venues.query('`Venue Category`.isnull().values | Venue.isnull().values')
pick

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1871,"St. James Town, Cabbagetown",43.667967,-79.367675,None,NaN,NaN,None


Ok, let's query the API again.

In [206]:
pick_venues = getNearbyVenues(pick['Neighborhood'], pick['Neighborhood Latitude'], pick['Neighborhood Longitude'],
                              radius=500)
pick_venues.head()

St. James Town, Cabbagetown


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"St. James Town, Cabbagetown",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
1,"St. James Town, Cabbagetown",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
2,"St. James Town, Cabbagetown",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant
3,"St. James Town, Cabbagetown",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Merryberry Cafe + Bistro,43.666630,-79.368792,Café


Good! Now we have values for all the neighborhoods! Let's drop the empty row(s) and append the new rows.

In [216]:
toronto_venues_fixed = toronto_venues.drop(pick.index).append(pick_venues, ignore_index=True)
assert( toronto_venues.shape[0] - 1 + pick_venues.shape[0] == toronto_venues_fixed.shape[0] )

Let's order neighborhoods by the **number of venues** returned (up to the maximum limit we chose).

In [227]:
toronto_venues_fixed.groupby('Neighborhood')[['Venue']].count().sort_values('Venue', ascending=False)

,Venue
Neighborhood,
"First Canadian Place, Underground city",100
"Harbourfront East, Union Station, Toronto Islands",100
"Garden District, Ryerson",100
"Commerce Court, Victoria Hotel",100
"Toronto Dominion Centre, Design Exchange",100
Enclave of M5E,100
"Richmond, Adelaide, King",94
St. James Town,81
"Fairview, Henry Farm, Oriole",74


Let's one-hot encode the venue categories in order to start preparing for neighborhood clustering. First let's check what are the different venues.

In [240]:
toronto_venues_fixed['Venue Category'].value_counts()

Coffee Shop                        181
Café                                91
Restaurant                          68
Park                                57
Pizza Place                         48
Italian Restaurant                  45
Bakery                              44
Japanese Restaurant                 42
Sandwich Place                      40
Hotel                               38
Clothing Store                      34
Gym                                 33
Sushi Restaurant                    32
Fast Food Restaurant                31
Grocery Store                       29
Bank                                28
Bar                                 27
Pub                                 24
Thai Restaurant                     24
Breakfast Spot                      22
Seafood Restaurant                  22
Pharmacy                            22
American Restaurant                 21
Gastropub                           18
Deli / Bodega                       18
Bookstore                

As it turns out there is a **Neighborhood** venue category in there! 

In [242]:
toronto_venues_fixed.query('`Venue Category` == "Neighborhood"')

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
345,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
590,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
781,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
1262,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


This is going to mess up the downstream analysis, where we already rely on the `Neighborhood` column for grouping, so let's replace these values with **Neighboorhood Area**.

In [248]:
toronto_venues_fixed['Venue Category'].replace({'Neighborhood': 'Neighborhood Area'}, inplace=True)
toronto_venues_fixed[ toronto_venues_fixed['Venue Category'].str.contains('Neigh') ]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
345,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood Area
590,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood Area
781,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood Area
1262,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood Area


In [324]:
features = pd.get_dummies(toronto_venues_fixed[['Venue Category']], prefix='', prefix_sep='')
features = pd.concat([toronto_venues_fixed[['Neighborhood']], features], axis=1)
features.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood Area,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Great! Let's compute the venue frequencies for each neighborhood and then identify the top 10 venues per neighborhood.

In [325]:
features_freqs = features.groupby('Neighborhood').mean().reset_index()

In [326]:
def getTop10(row):
    #  get the neighborhood venue frequencies as a single row
    #  order them in descending order
    #  take the top10
    p = row.iloc[0].sort_values(ascending=False).iloc[:10].reset_index().set_axis(['top venue', 'freq'], axis=1)
    
    #  make sure zero frequency venues do not count in the top10 list
    p.loc[ p.freq.eq(0.0), 'top venue' ] = None
    
    #  rename the index as Top1, Top2, ..., Top10
    p['name'] = ['Top' + str(i) for i in range(1, 11)]
    p.set_index('name', inplace=True)
    
    return p['top venue']

#  ok, let's run this function now neighborhood by neighborhood on our DataFrame
top10 = features_freqs.groupby('Neighborhood').apply(getTop10)
top10.head()

name,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10
Neighborhood,,,,,,,,,,
Agincourt,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Clothing Store,None,None,None,None,None
"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Sandwich Place,Coffee Shop,Pub,None,None,None,None
"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Health Food Store,Diner,Shopping Mall,Middle Eastern Restaurant,Mobile Phone Shop,Sandwich Place,Fried Chicken Joint,Supermarket
Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,None,None,None,None,None,None
"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Japanese Restaurant,Toy / Game Store,Juice Bar,Sushi Restaurant,Pub,Liquor Store,Thai Restaurant


# Cluster the Toronto Neighborhoods

In [327]:
from sklearn.cluster import KMeans

Let's try to cluster neighborhoods into four major clusters. For than we need to summarize the one-hot encoding of each neighborhood into a **sparse one-hot** version where each neighborhood has a single row of one-hot values.

In [334]:
n_clusters = 3
features_sparse = features.groupby('Neighborhood').sum().reset_index()
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(features_sparse.drop('Neighborhood', axis=1))
features_sparse.insert(1, column='cluster', value=kmeans.labels_)

In [335]:
features_sparse.head()

,Neighborhood,cluster,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood Area,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

In [336]:
features_sparse.cluster.value_counts()

2    78
0    16
1     6
Name: cluster, dtype: int64

Ok! The cluster representation looks relatively ok, let's create an annotated map of neighborhoods!

In [341]:
toronto_clustered = toronto.loc[ toronto.Neighborhood.isin(features_sparse.Neighborhood) ]\
.set_index('Neighborhood').join(features_sparse.set_index('Neighborhood'), on='Neighborhood').reset_index()
toronto_clustered.head()

,Neighborhood,PostalCode,Borough,Latitude,Longitude,cluster,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood Area,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,M3A,North York,43.753259,-79.329656,2,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [350]:
toronto_map = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=11)

cluster_color = {0: 'red', 1: 'steelblue', 2: 'green'}

for lat, lng, neighborhood, cluster in zip(toronto_clustered.Latitude, 
                                           toronto_clustered.Longitude, 
                                           toronto_clustered.Neighborhood,
                                           toronto_clustered.cluster):
    label = folium.Popup(str(neighborhood) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=cluster_color[cluster],
        fill=True,
        fill_color=cluster_color[cluster],
        fill_opacity=1.0,
        parse_html=False).add_to(toronto_map)
toronto_map